### --- Day 9: Disk Fragmenter ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/9

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 9);
WriteLines(inputLines, maxCols: 80);

Loading puzzle file: Day9.txt
Total lines: 1
Max line length: 19999

37177921644951938277999269384375694569563018897314838836721412555971912060537033


In [4]:
var inputLine = inputLines[0];

In [5]:
var testInputLine = "2333133121414131402";

Whoa, that puzzle input looks like it could render into an impossibly huge buffer! Let's see exactly how big it would be...

In [6]:
int CalcBufferSize(string inputLine) => inputLine.ToCharArray().Select(ToInt).Sum();

Console.WriteLine(CalcBufferSize(testInputLine));
Console.WriteLine(CalcBufferSize(inputLine));

42
95186


Ok, 95K is not too bad. I think the simplest approach will be to maintain pointers at each end of a buffer, and move items from the end to the start until the pointers cross over.

In [7]:
long DefragAndChecksum(string inputLine)
{
    var buffer = CreateBuffer(inputLine);
    Defrag(buffer);
    return CalculateChecksum(buffer);
}

// Use 0 to represent empty. This creates a slight problem as the file IDs also
// start at 0. But we'll mitigate that by starting the file IDs at 1 and correcting
// them during the checksum
const int EMPTY = 0;

int[] CreateBuffer(string inputLine)
{
    var bufferSize = CalcBufferSize(inputLine);
    var buffer = new int[bufferSize];

    var inputInts = inputLine.ToCharArray().Select(ToInt);

    var fileId = 1;
    var pointer = 0;

    void writeFile(int length)
    {
        foreach (var i in Enumerable.Range(0, length))
        {
            buffer[pointer++] = fileId;
        }
        fileId++;
    }

    foreach (var (index, length) in inputInts.Index())
    {
        if (index % 2 == 0)
        {
            writeFile(length);
        }
        else
        {
            // Empty space, already 0
            pointer += length;
        }
    }

    return buffer;
}

void Defrag(int[] fileBuffer)
{
    int rightPointer = fileBuffer.Length - 1;
    int leftPointer = 0;

    SafetyLimit safetyLimit = new();

    while (true)
    {
        safetyLimit.EnsureBelow(50_000);

        while (fileBuffer[rightPointer] == EMPTY)
        {
            rightPointer--;
        }
        while (fileBuffer[leftPointer] != EMPTY)
        {
            leftPointer++;
        }

        if (leftPointer >= rightPointer)
        {
            return;
        }

        // Swap
        fileBuffer[leftPointer] = fileBuffer[rightPointer];
        fileBuffer[rightPointer] = EMPTY;
    }
}

long CalculateChecksum(int[] fileBuffer) => fileBuffer.Where(i => i != EMPTY).Select((item, i) => i * (long)(item - 1)).Sum();

In [8]:
// ...In this example, the checksum is the sum of these, 1928.

var testAnswer = DefragAndChecksum(testInputLine);
Console.WriteLine(testAnswer);

1928


In [9]:
// Compact the amphipod's hard drive using the process he requested. What is the
// resulting filesystem checksum? (Be careful copy/pasting the input for this
// puzzle; it is a single, very long line.)

var part1Answer = DefragAndChecksum(inputLine);
Console.WriteLine(part1Answer);

6421128769094


In [10]:
// 6421128769094 is correct!
Ensure(6421128769094, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/9

Ok, I think for this part we'll have to switch to linked lists, so we can keep the blocks grouped together

In [12]:
abstract record DiskElement(int Length) 
{
    public abstract string ToRenderString();
}

record FreeSpace(int Length) : DiskElement(Length)
{
    public override string ToRenderString() => new string(Enumerable.Range(0, Length).Select(_ => '.').ToArray());
}

record File(int Length, int FileId) : DiskElement(Length)
{
    public override string ToRenderString() => string.Join("", Enumerable.Range(0, Length).Select(_ => FileId.ToString()));
}

In [13]:
using Disk = SCG.LinkedList<DiskElement>;
using ElementNode = SCG.LinkedListNode<DiskElement>;
using FreeNode = SCG.LinkedListNode<FreeSpace>;
using FileNode = SCG.LinkedListNode<File>;

Disk CreateDisk(string inputLine)
{
    var digits = inputLine.Select(ToInt);

    Disk result = new();

    int fileId = 0;
    foreach (var (index, length) in digits.Index())
    {
        DiskElement elem = index switch {
            var x when x % 2 == 0 => new File(length, fileId++),
            _ => new FreeSpace(length)
        };
        result.AddLast(elem);
    }

    return result;
}

string Render(Disk disk) => string.Join("", disk.Select(elem => elem.ToRenderString()));

// Test

void TestCreateDisk(string inputLine)
{
    var disk = CreateDisk(inputLine);
    var testRender = Render(disk);
    Console.WriteLine(testRender);
    Ensure("00...111...2...333.44.5555.6666.777.888899", testRender);
}
TestCreateDisk(testInputLine);

00...111...2...333.44.5555.6666.777.888899


Ok, within our `Disk` linked list, we have all the files and free space linked together, in order.

To defrag, pick file nodes off the end of the list, then search forwards from the start of the list until we find a free space node big enough to fit the file. Potentially this could be made more efficient by using some kind of free-list, which groups the free spaces by size, so we don't search the entire list from scratch every time, but hopefully this basic algorithm is fast enough for the given input.

Once we have found our free space in which to insert, we insert the file before the free space node, then truncate the length of the free space node, or delete it if the file fits exactly.

In [14]:
void Defrag2(Disk disk)
{
    var filesToMove = disk.WalkNodes().Where(n => n.Value is File).Reverse().ToList();

    foreach (var fileToMove in filesToMove)
    {
        var freeSpaceNode = disk.WalkNodes()
            .TakeWhile(n => n != fileToMove)
            .Where(n => n.Value is FreeSpace fs && fs.Length >= fileToMove.Value.Length)
            .FirstOrDefault();
        if (freeSpaceNode is null)
        {
            continue;
        }

        // Put some free space where the file used to be
        disk.AddAfter(fileToMove, new FreeSpace(fileToMove.Value.Length));
        disk.Remove(fileToMove);

        // Move the file ahead of the free space node
        disk.AddBefore(freeSpaceNode, fileToMove);
        var remainLength = freeSpaceNode.Value.Length - fileToMove.Value.Length;
        // Truncate or delete the free space
        if (remainLength > 0) {
            freeSpaceNode.Value = new FreeSpace(remainLength);
        } else {
            disk.Remove(freeSpaceNode);
        }
    }
}

// Test

void TestDefrag2(string inputLine)
{
    var disk = CreateDisk(inputLine);
    Defrag2(disk);
    var testRender = Render(disk);
    Console.WriteLine(testRender);
    Ensure("00992111777.44.333....5555.6666.....8888..", testRender);
}
TestDefrag2(testInputLine);

00992111777.44.333....5555.6666.....8888..


In [15]:
long CalculateChecksum2(Disk d)
{
    long sum = 0;

    int i = 0;
    foreach (var node in d.WalkNodes())
    {
        if (node.Value is File f)
        {
            sum += Enumerable.Range(i, f.Length).Select(j => (long)j * f.FileId).Sum();
        }
        i += node.Value.Length;
    }

    return sum;
}

In [16]:
// Put it all together

long DefragAndChecksum2(string inputLine)
{
    var disk = CreateDisk(inputLine);
    Defrag2(disk);
    return CalculateChecksum2(disk);
}

In [17]:
// The process of updating the filesystem checksum is the same; now, this
// example's checksum would be 2858.

var part2TestAnswer = DefragAndChecksum2(testInputLine);
Console.WriteLine(part2TestAnswer);

2858


In [18]:
// Start over, now compacting the amphipod's hard drive using this new method
// instead. What is the resulting filesystem checksum?

var part2Answer = DefragAndChecksum2(inputLine);
Console.WriteLine(part2Answer);

6448168620520


In [19]:
// 6448168620520 is correct!
Ensure(6448168620520, part2Answer);